## Assignment: Train GBM with Cross Validation

### University of California, Santa Barbara  
### PSTAT 135/235  
### Last Updated: Oct 31, 2018

---  

### MODULES

In [1]:
import os
import pandas as pd

In [2]:
import pyspark.sql.functions as F
import pyspark.mllib.regression as reg
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.mllib.linalg import SparseVector, DenseVector
from pyspark.sql.types import *

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local") \
    .appName("data preprocessing") \
    .config("spark.executor.memory", '8g') \
    .config('spark.executor.cores', '4') \
    .config('spark.cores.max', '4') \
    .config("spark.driver.memory",'8g') \
    .getOrCreate()

sc = spark.sparkContext

### PARAMETERS

In [2]:
path_to_data = os.path.join('/home/jovyan/UCSB_BigDataAnalytics/data/brca/breast_cancer_wisconsin.csv')

NameError: name 'os' is not defined

In [ ]:
# class = 2 for benign (negative class, 4 for malignant (positive class)
target = 'class'
positive_label = 4
negative_label = 2

SEED = 314
training_fraction = 0.6
ITERS = 10
FOLDS = 5

### READ IN DATA

In [ ]:
brca = spark.read.csv(path_to_data, header=True, inferSchema=True)

In [ ]:
brca.printSchema()

In [ ]:
print('rows={},columns={}'.format(brca.count(),len(brca.columns)))

In [3]:
# compute distribution of target variable
brca.groupBy(target).count().show()

In [5]:
# map target labels to 0/1

brca = brca.withColumn(target,F.when(brca[target] == positive_label, 1).otherwise(0))

In [11]:
data_train, data_test = brca.randomSplit([training_fraction, 1 - training_fraction], seed=SEED)

In [ ]:
print('records_total={},records_train={},records_test={}'.format(brca.count(),data_train.count(),data_test.count()))

In [6]:
vars_to_keep = [
 'clump_thickness',
 'uniformity_cell_size',
 'uniformity_cell_shape',
 'marginal_adhesion',
 'single_epithelial_cell_size'
]

In [ ]:
assembler = VectorAssembler(
                            inputCols=[c for c in data_train.columns if c in vars_to_keep],
                            outputCol='features')

In [ ]:
data_train = (assembler.transform(data_train).select(target, "features"))

In [ ]:
data_train.show(3)

In [17]:
gbt = GBTClassifier(labelCol=target, featuresCol="features", maxIter=ITERS)

In [18]:
evaluator = BinaryClassificationEvaluator(labelCol=target)

In [19]:
paramGrid = ParamGridBuilder() \
            .addGrid(gbt.maxIter, [1, 5]) \
            .addGrid(gbt.maxDepth, [1, 2]) \
            .build()

In [20]:
# k-fold cross validation
crossval = CrossValidator(
            estimator=gbt, 
            estimatorParamMaps=paramGrid, 
            evaluator=evaluator, 
            numFolds=FOLDS)

In [21]:
model = crossval.fit(data_train)

In [81]:
predictions = model.transform(
    assembler.transform(data_test).select("features"))

In [22]:
model.bestModel

GBTClassificationModel (uid=GBTClassifier_46cdbda6c695cf72bce1) with 5 trees

In [173]:
feat_imp = DenseVector(model.bestModel.featureImportances); feat_imp

DenseVector([0.0791, 0.1913, 0.3531, 0.1058, 0.2706])

In [174]:
# pack variables, feature importances into pandas dataframe

df_feat_imp = pd.DataFrame(index=vars_to_keep, columns=['feature_importance'], data=feat_imp[:])
df_feat_imp.sort_values(by='feature_importance', ascending=False, inplace=True)

In [ ]:
df_feat_imp